In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


path = "./drive/MyDrive/VietHung_PBL6_demo/Data_Image"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

Image_dataset = datasets.ImageFolder(root=path, transform=transform)


In [ ]:
# Chia dữ liệu thành tập train và tập test
train_dataset, test_dataset = random_split(Image_dataset, [0.7, 0.3])

batch_size = 32
# Tạo DataLoader để nạp dữ liệu theo batch
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
len(Image_dataset)

1346

In [2]:
from torchvision.models import resnet18
import torch.nn as nn

class Vison_model(nn.Module):
    def __init__(self, num_classes=16):
        super(Vison_model, self).__init__()

        self.resnet18 = resnet18(pretrained=False)

        modelOutputFeats = self.resnet18.fc.in_features

        self.resnet18.fc = nn.Sequential(
            nn.Linear(modelOutputFeats,num_classes),
            nn.LogSoftmax(dim=1),
            )


    def forward(self, x):
        # Forward pass through VGG16
        x = self.resnet18(x)
        return x


num_classes = len(Image_dataset.classes)
vision_model = Vison_model(num_classes)


In [3]:
resnet18 = resnet18(pretrained=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import torch

def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn):
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(data_loader):
        print(X.shape, y.shape)
        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        print(loss,y.shape,y_pred.shape)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for (X, y) in data_loader:
            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(vision_model.parameters(), lr=0.0001)
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
torch.manual_seed(42)

epochs = 30
for epoch in range(epochs):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_loader,
        model=vision_model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_step(data_loader=test_loader,
        model=vision_model,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )


Epoch: 0
---------
torch.Size([32, 3, 224, 224]) torch.Size([32])
tensor(2.9166, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32])
tensor(2.9459, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32])
tensor(2.6711, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32])
tensor(2.7172, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32])
tensor(2.6072, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32])
tensor(2.3063, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32])
tensor(2.4232, grad_fn=<NllLossBackward0>) torch.Size([32]) torch.Size([32, 16])
torch.Size([32, 3, 224, 224]) torch.Size([32])
tensor(2.6332, grad_fn=<NllLossBackwar